In [1]:
ENV_NAME = 'Safexp-CarGoal1-v0'
alias = 'ET_Context_Penalty1'

import safety_gym
import gym
print('all benchmarking environments:')
print('\n now evaluating: \n       ', ENV_NAME)


env = gym.make(ENV_NAME)

from collections import deque
from context_buffer import Buffers
import matplotlib.pyplot as plt
import numpy as np
import torch
import argparse
import os
import torch.nn.functional as F
import utils
import TD3_Context
import OurDDPG
import DDPG

def eval_policy(policy, eval_episodes=10):
    eval_env = gym.make(ENV_NAME)
    avg_reward = 0.
    avg_cost = 0.
    for _ in range(eval_episodes):
        state, done = eval_env.reset(), False
        rewards_hist = deque(maxlen = args_hist_len)
        actions_hist = deque(maxlen = args_hist_len)
        states_hist = deque(maxlen = args_hist_len)

        next_hrews = deque(maxlen = args_hist_len)
        next_hacts = deque(maxlen = args_hist_len)
        next_hstates = deque(maxlen = args_hist_len)

        zero_action = np.zeros(env.action_space.shape[0])
        zero_state = np.zeros(state.shape)

        for _ in range(args_hist_len):
            rewards_hist.append(0)
            actions_hist.append(zero_action.copy())
            states_hist.append(zero_state.copy())

            next_hrews.append(0)
            next_hacts.append(zero_action.copy())
            next_hstates.append(zero_state.copy())

        rand_acttion = np.random.normal(0, args_expl_noise, size=env.action_space.shape[0])
        rand_acttion = rand_acttion.clip(env.action_space.low, env.action_space.high)
        rewards_hist.append(0)
        actions_hist.append(rand_acttion.copy())
        states_hist.append(state.copy())


        while not done:
            np_pre_actions = np.asarray(actions_hist, dtype=np.float32).flatten()  #(hist, action_dim) => (hist *action_dim,)
            np_pre_rewards = np.asarray(rewards_hist, dtype=np.float32) #(hist, )
            np_pre_obsers = np.asarray(states_hist, dtype=np.float32).flatten()  #(hist, action_dim) => (hist *action_dim,)

            action = (
                    policy.select_action(np.array(state),np.array(np_pre_actions), np.array(np_pre_rewards), np.array(np_pre_obsers))
                    + 0.0
                ).clip(-max_action, max_action)
            next_state, reward, done,_ = eval_env.step(action)
            avg_cost += _['cost']
            avg_reward += reward
            
            rewards_hist.append(reward)
            actions_hist.append(action.copy())
            states_hist.append(state.copy())
            state = next_state
            
    avg_reward /= eval_episodes
    avg_cost /= eval_episodes
    return avg_reward,avg_cost

def eval_policy_ET(policy, eval_episodes=10):
    eval_env = gym.make(ENV_NAME)
    avg_reward = 0.
    avg_cost = 0.
    for _ in range(eval_episodes):
        state, done = eval_env.reset(), False
        rewards_hist = deque(maxlen = args_hist_len)
        actions_hist = deque(maxlen = args_hist_len)
        states_hist = deque(maxlen = args_hist_len)

        next_hrews = deque(maxlen = args_hist_len)
        next_hacts = deque(maxlen = args_hist_len)
        next_hstates = deque(maxlen = args_hist_len)

        zero_action = np.zeros(env.action_space.shape[0])
        zero_state = np.zeros(state.shape)

        for _ in range(args_hist_len):
            rewards_hist.append(0)
            actions_hist.append(zero_action.copy())
            states_hist.append(zero_state.copy())

            # same thing for next_h*
            next_hrews.append(0)
            next_hacts.append(zero_action.copy())
            next_hstates.append(zero_state.copy())

        rand_acttion = np.random.normal(0, args_expl_noise, size=env.action_space.shape[0])
        rand_acttion = rand_acttion.clip(env.action_space.low, env.action_space.high)
        rewards_hist.append(0)
        actions_hist.append(rand_acttion.copy())
        states_hist.append(state.copy())


        while not done:
            np_pre_actions = np.asarray(actions_hist, dtype=np.float32).flatten()  #(hist, action_dim) => (hist *action_dim,)
            np_pre_rewards = np.asarray(rewards_hist, dtype=np.float32) #(hist, )
            np_pre_obsers = np.asarray(states_hist, dtype=np.float32).flatten()  #(hist, action_dim) => (hist *action_dim,)

            action = (
                    policy.select_action(np.array(state),np.array(np_pre_actions), np.array(np_pre_rewards), np.array(np_pre_obsers))
                    + 0.0
                ).clip(-max_action, max_action)
            next_state, reward, done,_ = eval_env.step(action)
            avg_cost += _['cost']
            if _['cost'] == 1:
                done = True
            avg_reward += reward
            
            rewards_hist.append(reward)
            actions_hist.append(action.copy())
            states_hist.append(state.copy())
            state = next_state
            
    avg_reward /= eval_episodes
    avg_cost /= eval_episodes
    return avg_reward,avg_cost


def eval_policy_WOS(policy, eval_episodes=10):
    eval_env = gym.make(ENV_NAME)
    avg_reward = 0.
    avg_cost = 0.
    for _ in range(eval_episodes):
        state, done = eval_env.reset(), False
        rewards_hist = deque(maxlen = args_hist_len)
        actions_hist = deque(maxlen = args_hist_len)
        states_hist = deque(maxlen = args_hist_len)

        next_hrews = deque(maxlen = args_hist_len)
        next_hacts = deque(maxlen = args_hist_len)
        next_hstates = deque(maxlen = args_hist_len)

        zero_action = np.zeros(env.action_space.shape[0])
        zero_state = np.zeros(state.shape)

        for _ in range(args_hist_len):
            rewards_hist.append(0)
            actions_hist.append(zero_action.copy())
            states_hist.append(zero_state.copy())

            # same thing for next_h*
            next_hrews.append(0)
            next_hacts.append(zero_action.copy())
            next_hstates.append(zero_state.copy())

        rand_acttion = np.random.normal(0, args_expl_noise, size=env.action_space.shape[0])
        rand_acttion = rand_acttion.clip(env.action_space.low, env.action_space.high)
        rewards_hist.append(0)
        actions_hist.append(rand_acttion.copy())
        states_hist.append(state.copy())


        while not done:
            np_pre_actions = np.asarray(actions_hist, dtype=np.float32).flatten()  #(hist, action_dim) => (hist *action_dim,)
            np_pre_rewards = np.asarray(rewards_hist, dtype=np.float32) #(hist, )
            np_pre_obsers = np.asarray(states_hist, dtype=np.float32).flatten()  #(hist, action_dim) => (hist *action_dim,)

            action = (
                    policy.select_action(np.array(state),np.array(np_pre_actions), np.array(np_pre_rewards), np.array(np_pre_obsers))
                    + 0.0
                ).clip(-max_action, max_action)
            next_state, reward, done,_ = eval_env.step(action)
            avg_cost += _['cost']
            avg_reward += reward
            
            rewards_hist.append(reward)
            actions_hist.append(action.copy())
            states_hist.append(zero_state.copy())
            state = next_state
            
    avg_reward /= eval_episodes
    avg_cost /= eval_episodes
    return avg_reward,avg_cost


def eval_policy_WOA(policy, eval_episodes=10):
    eval_env = gym.make(ENV_NAME)
    avg_reward = 0.
    avg_cost = 0.
    for _ in range(eval_episodes):
        state, done = eval_env.reset(), False
        rewards_hist = deque(maxlen = args_hist_len)
        actions_hist = deque(maxlen = args_hist_len)
        states_hist = deque(maxlen = args_hist_len)

        next_hrews = deque(maxlen = args_hist_len)
        next_hacts = deque(maxlen = args_hist_len)
        next_hstates = deque(maxlen = args_hist_len)

        zero_action = np.zeros(env.action_space.shape[0])
        zero_state = np.zeros(state.shape)

        for _ in range(args_hist_len):
            rewards_hist.append(0)
            actions_hist.append(zero_action.copy())
            states_hist.append(zero_state.copy())

            # same thing for next_h*
            next_hrews.append(0)
            next_hacts.append(zero_action.copy())
            next_hstates.append(zero_state.copy())

        rand_acttion = np.random.normal(0, args_expl_noise, size=env.action_space.shape[0])
        rand_acttion = rand_acttion.clip(env.action_space.low, env.action_space.high)
        rewards_hist.append(0)
        actions_hist.append(rand_acttion.copy())
        states_hist.append(state.copy())


        while not done:
            np_pre_actions = np.asarray(actions_hist, dtype=np.float32).flatten()  #(hist, action_dim) => (hist *action_dim,)
            np_pre_rewards = np.asarray(rewards_hist, dtype=np.float32) #(hist, )
            np_pre_obsers = np.asarray(states_hist, dtype=np.float32).flatten()  #(hist, action_dim) => (hist *action_dim,)
            action = (
                    policy.select_action(np.array(state),np.array(np_pre_actions), np.array(np_pre_rewards), np.array(np_pre_obsers))
                    + 0.0
                ).clip(-max_action, max_action)
            next_state, reward, done,_ = eval_env.step(action)
            avg_cost += _['cost']
            avg_reward += reward
            
            rewards_hist.append(reward)
            actions_hist.append(zero_action.copy())
            states_hist.append(state.copy())
            state = next_state
            
    avg_reward /= eval_episodes
    avg_cost /= eval_episodes
    return avg_reward,avg_cost

def eval_policy_WOR(policy, eval_episodes=10):
    eval_env = gym.make(ENV_NAME)
    avg_reward = 0.
    avg_cost = 0.
    for _ in range(eval_episodes):
        state, done = eval_env.reset(), False
        rewards_hist = deque(maxlen = args_hist_len)
        actions_hist = deque(maxlen = args_hist_len)
        states_hist = deque(maxlen = args_hist_len)

        next_hrews = deque(maxlen = args_hist_len)
        next_hacts = deque(maxlen = args_hist_len)
        next_hstates = deque(maxlen = args_hist_len)

        zero_action = np.zeros(env.action_space.shape[0])
        zero_state = np.zeros(state.shape)

        for _ in range(args_hist_len):
            rewards_hist.append(0)
            actions_hist.append(zero_action.copy())
            states_hist.append(zero_state.copy())

            # same thing for next_h*
            next_hrews.append(0)
            next_hacts.append(zero_action.copy())
            next_hstates.append(zero_state.copy())

        rand_acttion = np.random.normal(0, args_expl_noise, size=env.action_space.shape[0])
        rand_acttion = rand_acttion.clip(env.action_space.low, env.action_space.high)
        rewards_hist.append(0)
        actions_hist.append(rand_acttion.copy())
        states_hist.append(state.copy())


        while not done:
            np_pre_actions = np.asarray(actions_hist, dtype=np.float32).flatten()  #(hist, action_dim) => (hist *action_dim,)
            np_pre_rewards = np.asarray(rewards_hist, dtype=np.float32) #(hist, )
            np_pre_obsers = np.asarray(states_hist, dtype=np.float32).flatten()  #(hist, action_dim) => (hist *action_dim,)
            action = (
                    policy.select_action(np.array(state),np.array(np_pre_actions), np.array(np_pre_rewards), np.array(np_pre_obsers))
                    + 0.0
                ).clip(-max_action, max_action)
            next_state, reward, done,_ = eval_env.step(action)
            avg_cost += _['cost']
            avg_reward += reward
            
            rewards_hist.append(0)
            actions_hist.append(action.copy())
            states_hist.append(state.copy())
            state = next_state
            
    avg_reward /= eval_episodes
    avg_cost /= eval_episodes
    return avg_reward,avg_cost


env = gym.make(ENV_NAME)
torch.manual_seed(0)
np.random.seed(0)

state_dim = env.observation_space.shape[0]
action_dim = env.action_space.shape[0]
max_action = env.action_space.high[0]

args_policy_noise = 0.2
args_noise_clip = 0.5
args_policy_freq = 2
args_max_timesteps = 1000000
args_expl_noise = 0.1
args_batch_size = 256
args_eval_freq = 1000
args_start_timesteps = 25000
args_hist_len = 7

kwargs = {
    "state_dim": state_dim,
    "action_dim": action_dim,
    "max_action": max_action,
    "discount": 0.99,
    "tau": 0.005,
    "hist_len":args_hist_len
}

for repeat in range(10,11):
    if True:
        args_policy = 'Context_TD3'

        if args_policy == "Context_TD3":
            # Target policy smoothing is scaled wrt the action scale
            kwargs["policy_noise"] = args_policy_noise * max_action
            kwargs["noise_clip"] = args_noise_clip * max_action
            kwargs["policy_freq"] = args_policy_freq
            policy = TD3_Context.TD3(**kwargs)
        elif args_policy == "OurDDPG":
            policy = OurDDPG.DDPG(**kwargs)
        elif args_policy == "DDPG":
            policy = DDPG.DDPG(**kwargs)
    #replay_buffer = utils.ReplayBuffer(state_dim, action_dim)
    replay_buffer = Buffer(max_size = 1e6)
    # Evaluate untrained policy
    evaluations = [eval_policy(policy)]
    evaluations_ET = [eval_policy_ET(policy)]
    evaluations_WOR = [eval_policy_WOR(policy)]
    evaluations_WOA = [eval_policy_WOA(policy)]
    evaluations_WOS = [eval_policy_WOS(policy)]

    
    
    '''initialization of the training process'''
    state, done = env.reset(), False
    
    episode_reward = 0
    episode_timesteps = 0
    episode_num = 0
    counter = 0
    msk_list = []        
    temp_curve = [eval_policy(policy)]
    temp_val = []
    
    rewards_hist = deque(maxlen = args_hist_len)
    actions_hist = deque(maxlen = args_hist_len)
    states_hist = deque(maxlen = args_hist_len)
    
    next_hrews = deque(maxlen = args_hist_len)
    next_hacts = deque(maxlen = args_hist_len)
    next_hstates = deque(maxlen = args_hist_len)
    
    zero_action = np.zeros(env.action_space.shape[0])
    zero_state = np.zeros(state.shape)
    
    for _ in range(args_hist_len):
        rewards_hist.append(0)
        actions_hist.append(zero_action.copy())
        states_hist.append(zero_state.copy())

        # same thing for next_h*
        next_hrews.append(0)
        next_hacts.append(zero_action.copy())
        next_hstates.append(zero_state.copy())
    
    rand_acttion = np.random.normal(0, args_expl_noise, size=env.action_space.shape[0])
    rand_acttion = rand_acttion.clip(env.action_space.low, env.action_space.high)
    rewards_hist.append(0)
    actions_hist.append(rand_acttion.copy())
    states_hist.append(state.copy())
    
    for t in range(int(args_max_timesteps)):
        episode_timesteps += 1
        counter += 1
        # Select action randomly or according to policy
        
        
        np_pre_actions = np.asarray(actions_hist, dtype=np.float32).flatten()  #(hist, action_dim) => (hist *action_dim,)
        np_pre_rewards = np.asarray(rewards_hist, dtype=np.float32) #(hist, )
        np_pre_obsers = np.asarray(states_hist, dtype=np.float32).flatten()  #(hist, action_dim) => (hist *action_dim,)
        
        if t < args_start_timesteps:
            action = np.random.uniform(-max_action,max_action,action_dim)
        else:
            if np.random.uniform(0,1) < 0.0:
                action = np.random.uniform(-max_action,max_action,action_dim)
            else:
                action = (
                    policy.select_action(np.array(state),np.array(np_pre_actions), np.array(np_pre_rewards), np.array(np_pre_obsers))
                    + np.random.normal(0, max_action * args_expl_noise, size=action_dim)
                ).clip(-max_action, max_action)

        # Perform action
        state_temp = env.sim.get_state()
        next_state, reward, done,_ = env.step(action) 
        if _['cost'] == 1:
            print('danger! early termination!')
            reward -= 1
            done = True
            
        done_bool = float(done) if episode_timesteps < 1000 else 0
        
        episode_reward += reward
        next_hrews.append(reward)
        next_hacts.append(action.copy())
        next_hstates.append(state.copy())
        np_next_hacts = np.asarray(next_hacts, dtype=np.float32).flatten()  #(hist, action_dim) => (hist *action_dim,)
        np_next_hrews = np.asarray(next_hrews, dtype=np.float32) #(hist, )
        np_next_hobvs = np.asarray(next_hstates, dtype=np.float32).flatten() #(hist, )
        
        replay_buffer.add((state, next_state, action, reward, done_bool,
                           np_pre_actions, np_pre_rewards, np_pre_obsers,
                           np_next_hacts, np_next_hrews, np_next_hobvs))
        
        rewards_hist.append(reward)
        actions_hist.append(action.copy())
        states_hist.append(state.copy())
        
        state = next_state
        
        
        
        
        if t >= args_start_timesteps:
            policy.train_context_TD3(replay_buffer, args_batch_size)
                    
                    
        # Train agent after collecting sufficient data
        if done: 
            print(f"Total T: {t+1} Episode Num: {episode_num+1} Episode T: {episode_timesteps} Reward: {episode_reward:.3f}")
            msk_list = []
            state, done = env.reset(), False
            episode_reward = 0
            episode_timesteps = 0
            episode_num += 1 
            rewards_hist = deque(maxlen = args_hist_len)
            actions_hist = deque(maxlen = args_hist_len)
            states_hist = deque(maxlen = args_hist_len)

            next_hrews = deque(maxlen = args_hist_len)
            next_hacts = deque(maxlen = args_hist_len)
            next_hstates = deque(maxlen = args_hist_len)

            zero_action = np.zeros(env.action_space.shape[0])
            zero_state = np.zeros(state.shape)

            for _ in range(args_hist_len):
                rewards_hist.append(0)
                actions_hist.append(zero_action.copy())
                states_hist.append(zero_state.copy())

                # same thing for next_h*
                next_hrews.append(0)
                next_hacts.append(zero_action.copy())
                next_hstates.append(zero_state.copy())

            rand_acttion = np.random.normal(0, args_expl_noise, size=env.action_space.shape[0])
            rand_acttion = rand_acttion.clip(env.action_space.low, env.action_space.high)
            rewards_hist.append(0)
            actions_hist.append(rand_acttion.copy())
            states_hist.append(state.copy())
        # Evaluate episode
        if (t + 1) % args_eval_freq == 0:
            evaluations.append(eval_policy(policy))
            evaluations_ET.append(eval_policy_ET(policy))
            evaluations_WOR.append(eval_policy_WOR(policy))
            evaluations_WOA.append(eval_policy_WOA(policy))
            evaluations_WOS.append(eval_policy_WOS(policy))
            
            print('recent Evaluation:',evaluations[-1],
                  'recent Evaluation ET', evaluations_ET[-1],
                  'WOR',evaluations_WOR[-1],
                  'WOA',evaluations_WOA[-1], 
                  'WOS',evaluations_WOR[-1])
            np.save('results/evaluations_alias{}_ENV{}_Repeat{}'.format(alias,ENV_NAME,repeat),evaluations)
            np.save('results/evaluations_ET_alias{}_ENV{}_Repeat{}'.format(alias,ENV_NAME,repeat),evaluations_ET)
            np.save('results/evaluations_WOA_alias{}_ENV{}_Repeat{}'.format(alias,ENV_NAME,repeat),evaluations_WOA)
            np.save('results/evaluations_WOR_alias{}_ENV{}_Repeat{}'.format(alias,ENV_NAME,repeat),evaluations_WOR)
            np.save('results/evaluations_WOS_alias{}_ENV{}_Repeat{}'.format(alias,ENV_NAME,repeat),evaluations_WOS)

all benchmarking environments:

 now evaluating: 
        Safexp-CarGoal1-v0


/home/hsun/anaconda3/envs/open-mmlab/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
/home/hsun/anaconda3/envs/open-mmlab/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


danger! early termination!
Total T: 127 Episode Num: 1 Episode T: 127 Reward: -0.550
recent Evaluation: (-0.627656810454336, 2.4) recent Evaluation ET (-0.7131036576376929, 0.0) WOR (-0.7230141005638602, 3.9) WOA (-0.6503354132766612, 3.9) WOS (-0.7230141005638602, 3.9)
Total T: 1127 Episode Num: 2 Episode T: 1000 Reward: -1.457
danger! early termination!
Total T: 1282 Episode Num: 3 Episode T: 155 Reward: -0.563
recent Evaluation: (-0.8607022267242813, 0.0) recent Evaluation ET (-0.6908888834849856, 0.2) WOR (-0.818084863137423, 40.0) WOA (-0.6528358545289737, 17.2) WOS (-0.818084863137423, 40.0)
Total T: 2282 Episode Num: 4 Episode T: 1000 Reward: -2.634
recent Evaluation: (-0.7238692118210894, 0.0) recent Evaluation ET (-0.6284980811894211, 0.1) WOR (-0.879393034514948, 14.0) WOA (-0.6337288837401566, 5.5) WOS (-0.879393034514948, 14.0)
Total T: 3282 Episode Num: 5 Episode T: 1000 Reward: -1.567
danger! early termination!
Total T: 3531 Episode Num: 6 Episode T: 249 Reward: -0.276
re